# 重新实现之前的线性热模型
应该只需要读取第一次实验的数据，这里主要想规范一下之前的写法

In [ ]:
import pandas as pd
import numpy as np
import os
from keys import *
import math

from tqdm import tqdm
from loader import Loader
from thermal_model.data import *
from thermal_model.configs import *
from thermal_model.figure_plotter import *
from utils_thermal_model_raw_process import *
import matplotlib.pyplot as plt
from plotter import Plotter
import seaborn as sns
from sklearn.linear_model import LinearRegression,Lars
from sklearn.ensemble import RandomForestRegressor

# 读取数据并且缓存成单独pickle

In [ ]:
df_thermal_model_data_raw = ThermalModelData().load()

# 生成线性模型的输入项

In [ ]:
df_thermal_model_data_input = generate_model_input(df_thermal_model_data_raw)

In [ ]:
df_thermal_model_data_input.columns

# 生成模型（应该使用线性模型）

In [ ]:
model = RandomForestRegressor(
    n_estimators=10,
)

In [ ]:
model_input = df_thermal_model_data_input[thermal_model_input_cols]
model_output = df_thermal_model_data_input[thermal_model_output_cols]

In [ ]:
model.fit(
    X = model_input,
    y = model_output,
    
)

In [ ]:

model_predict = (model.predict(model_input))
error = np.squeeze(model_output) - model_predict
model.score(model_input,model_output)

In [ ]:
plt.scatter(
    model_output,
    model_predict
)

In [ ]:
_ = plt.hist(
    error,bins = 1000
)

In [ ]:
plt.plot(np.cumsum(np.squeeze(model_output)))
plt.plot(np.cumsum(np.squeeze(model_predict)))

In [ ]:
model.feature_importances_

# 测试模型功能

In [2]:
from thermal_model.configs import ElectrolyzerParameter

In [3]:
ep = ElectrolyzerParameter()

In [5]:
ep.active_surface_area

0.42498472674484183

In [6]:
from thermal_model.original_thermal_model_and_plotter_0117 import Electrolyzer as ele_ori
from thermal_model.electrolyzer import Electrolyzer


In [7]:
electrolyzer = Electrolyzer()
ele_ori_model = ele_ori()
ele_ori_model.merge_coef()

In [8]:
electrolyzer.pring_all_properties()

default_ambient_temperature :	 25
default_current :	 1500
default_lye_flow :	 1.2
default_lye_temperature :	 60
parameters :	 <thermal_model.configs.ElectrolyzerParameter object at 0x00000297B3588880>
num_cells :	 34
active_surface_area :	 0.42498472674484183
coefficient_electric_heat :	 1
coefficient_radiation_dissipation :	 -3.2736565035024594e-10
coefficient_input_lye_heat :	 0.5799335140794437
coefficient_output_lye_heat :	 -0.6565760564700911
coefficient_delta_temp :	 1023.6319772802835
interval :	 20


In [9]:
ele_ori_model.Print_All_Properties()

surf_area_active || 0.4254470362624
n_cell || 34
total_length || 0.6855
total_space || 0.35734932057370616
vacancy_space || 0.13597287278946305
occupied_space || 0.15038202349712662
rate_vacancy || 0.3805040753153399
surface_area || 4.2756352719702
interval || 20
max_current_density || 4000
epsilon || 0.95
weight_structural || 1188.0179856273003
weight_alkaline_inside || 174.0452771705127
weight_total || 1362.063262797813
HC_structural || 594.0089928136501
HC_alkaline_inside || 393.777439598285
HC_total || 987.7864324119352
HC_alkaline_in_flux || 0.8044444444444444
Model_Target || [1, -3.2736565035024594e-10, 0.5799335140794437, -0.6565760564700911, 1023.6319772802835]
corr_power || 1.0
corr_radiation || -1.4214310326772457
corr_HeatLyeIn || 0.7209118268943362
corr_HeatLyeOut || -0.8161857055567431
corr_DeltaTemp || 1.0362887600924242
coef_power || 1.0
coef_radiation || -3.27365650350246e-10
coef_HeatLyeIn || 0.5799335140794437
coef_HeatLyeOut || -0.6565760564700911
coef_DeltaTemp || 1

In [10]:
(
            current_list,
            voltage_list,
            power_list,
            temperature_list
        ) = electrolyzer.get_default_polarization()

In [14]:
electrolyzer.temperature_thermal_balance_current(
    ambient_temperature=15,
    lye_flow=1.5,
    lye_temperature=10,
    current=0
)

1.8645851828000517e-153